In [1]:
from Process import PlateReader
import pandas as pd

### Step 1: Load Sample Information
* Ideally will be in the form of a dataframe. This dataframe should be standardized in the original documentation where we originally made the sample. 
* Either load locally or pull from the google team drive (still working on fixing pulling from Drive)

In [15]:
sample_info = pd.read_csv(r"Hold_info")
sample_info = sample_info[0:96]

### Step 2: Load UV-Vis Data
* ***If loading from plate reader***: Once the plate has been read, allow for exporting to an excel sheet and reformat for easy upload and merging.
    * The plate reader should automatically ask you if you would like to export to excel. When running multiple plates you can just keep selecting export and it will automatically add the new run as an additional sheet to the originally generated excel sheet. 
    * To format for easy loading:
        1. Create a new sheet in the same exported excel file and name it something simple (i.e. sheet1). 
        2. Copy the data from the sheet and add it to the newly created sheet. Make sure to paste in the data as "only values". 
        3. Repeat for every single exported sheet (i.e. every plate)
         
* Sometimes when using the plate reader the scanning protocol scans the whole plate versus only the wells of interest. This can be changed, however it is common to forget. If you do forget to do this, make sure to trim the dataframe to match the same length/order of wells as the sample info loaded in Step 1.

* ***If loading from single cuvette reader***: 
    * Still documenting.

***Step 2a:*** Provide the path of the excel sheet for the plate reader. With this you will also need to provide the **Sheet Name** of for correct excel sheet to be extracted. Since multiple plates are expected to be used at once, provide the Sheet names as a ***list of string names***. This will return a list of dataframes of each plate in the order the sheet names provided.

In [16]:
path = r"C:\Users\Edwin\Downloads\04_07_21_PFO_New_Strategy.xlsx"
plate_names = ['Sheet1']
plate_dfs = PlateReader.extract_plates(path, plate_names)

***Step 2b:*** With the dataframes from ***Step2a***, given that all the plates read originally were done using the same protcol, then we can merge these dataframes into one. The only thing we need to edit is when originally formatting the excel sheets, since we copied the wavelength row in each sheet it only needs to be present once. A way to avoid to this is to copy it only in the first plate sheet you are using. 

Should be length of sample_info + 1 (will remove the wavelength column shortly).

In [17]:
merged_df = PlateReader.merge_wavelength_dfs(plate_dfs)
merged_df

,0,1,2,3,4,5,6,7,8,9,...,150,151,152,153,154,155,156,157,158,159
Wavelength,200.000,205.000,210.000,215.000,220.000,225.000,230.000,235.000,240.000,245.000,...,950.000,955.000,960.000,965.000,970.000,975.000,980.000,985.000,990.000,995.000
A1,3.373,3.433,3.405,3.385,3.428,3.466,3.737,3.510,3.451,3.544,...,0.122,0.143,0.159,0.167,0.171,0.174,0.173,0.172,0.170,0.164
A2,3.371,3.432,3.404,3.386,3.427,3.464,3.741,3.499,3.440,3.545,...,0.119,0.138,0.154,0.162,0.166,0.170,0.169,0.169,0.167,0.162
A3,3.360,3.420,3.392,3.377,3.422,3.462,3.739,3.509,3.451,3.548,...,0.109,0.128,0.142,0.150,0.154,0.158,0.158,0.158,0.156,0.152
A4,3.365,3.430,3.400,3.380,3.427,3.462,3.754,3.511,3.453,3.566,...,0.108,0.125,0.139,0.147,0.151,0.155,0.155,0.155,0.154,0.150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
H8,3.513,3.574,3.548,3.526,3.564,3.542,3.765,3.572,3.509,3.576,...,0.109,0.117,0.125,0.131,0.134,0.138,0.138,0.141,0.142,0.140
H9,3.482,3.538,3.516,3.497,3.524,3.556,3.739,3.543,3.482,3.556,...,0.105,0.113,0.120,0.125,0.129,0.133,0.133,0.136,0.138,0.136
H10,3.349,3.409,3.380,3.363,3.405,3.430,3.675,3.457,3.396,3.481,...,0.078,0.085,0.092,0.097,0.101,0.105,0.105,0.109,0.111,0.109
H11,3.479,3.538,3.511,3.493,3.529,3.530,3.735,3.551,3.475,3.555,...,0.104,0.112,0.119,0.124,0.128,0.132,0.132,0.135,0.136,0.134


***Step 2c:*** Sometimes when samples are extremely turbid or have some sort of material like foam or cream the plate reader absorbance is overflowed and returns as 'OVRFLW'. This will prevent you from handling the entire set of data, so what can be done is replace the 'OVRFLW' values with a holder number. It is reccomended you select a very high holder number so you can track and isolate it easier if needed. 

In [18]:
merged_df = PlateReader.detect_ovflw(merged_df, holder = 30)
merged_df

,0,1,2,3,4,5,6,7,8,9,...,150,151,152,153,154,155,156,157,158,159
Wavelength,200.000,205.000,210.000,215.000,220.000,225.000,230.000,235.000,240.000,245.000,...,950.000,955.000,960.000,965.000,970.000,975.000,980.000,985.000,990.000,995.000
A1,3.373,3.433,3.405,3.385,3.428,3.466,3.737,3.510,3.451,3.544,...,0.122,0.143,0.159,0.167,0.171,0.174,0.173,0.172,0.170,0.164
A2,3.371,3.432,3.404,3.386,3.427,3.464,3.741,3.499,3.440,3.545,...,0.119,0.138,0.154,0.162,0.166,0.170,0.169,0.169,0.167,0.162
A3,3.360,3.420,3.392,3.377,3.422,3.462,3.739,3.509,3.451,3.548,...,0.109,0.128,0.142,0.150,0.154,0.158,0.158,0.158,0.156,0.152
A4,3.365,3.430,3.400,3.380,3.427,3.462,3.754,3.511,3.453,3.566,...,0.108,0.125,0.139,0.147,0.151,0.155,0.155,0.155,0.154,0.150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
H8,3.513,3.574,3.548,3.526,3.564,3.542,3.765,3.572,3.509,3.576,...,0.109,0.117,0.125,0.131,0.134,0.138,0.138,0.141,0.142,0.140
H9,3.482,3.538,3.516,3.497,3.524,3.556,3.739,3.543,3.482,3.556,...,0.105,0.113,0.120,0.125,0.129,0.133,0.133,0.136,0.138,0.136
H10,3.349,3.409,3.380,3.363,3.405,3.430,3.675,3.457,3.396,3.481,...,0.078,0.085,0.092,0.097,0.101,0.105,0.105,0.109,0.111,0.109
H11,3.479,3.538,3.511,3.493,3.529,3.530,3.735,3.551,3.475,3.555,...,0.104,0.112,0.119,0.124,0.128,0.132,0.132,0.135,0.136,0.134


# Step 3: Merge with Sample Info Dataframe

The first step to merge to sample info dataframe is to format the plate reader dataframe to have the wavelengths as the column names versus as an another row. We can do this by isolating the Wavelength row, adding the appropiate unit the string of each element of the wavlength row and reassigning this new list as the column names. We finalize this by dropping the wavlength row.

In [19]:
platereader_df = PlateReader.rehead_wavelengths(merged_df)
platereader_df

,200.0nm,205.0nm,210.0nm,215.0nm,220.0nm,225.0nm,230.0nm,235.0nm,240.0nm,245.0nm,...,950.0nm,955.0nm,960.0nm,965.0nm,970.0nm,975.0nm,980.0nm,985.0nm,990.0nm,995.0nm
A1,3.373,3.433,3.405,3.385,3.428,3.466,3.737,3.510,3.451,3.544,...,0.122,0.143,0.159,0.167,0.171,0.174,0.173,0.172,0.170,0.164
A2,3.371,3.432,3.404,3.386,3.427,3.464,3.741,3.499,3.440,3.545,...,0.119,0.138,0.154,0.162,0.166,0.170,0.169,0.169,0.167,0.162
A3,3.360,3.420,3.392,3.377,3.422,3.462,3.739,3.509,3.451,3.548,...,0.109,0.128,0.142,0.150,0.154,0.158,0.158,0.158,0.156,0.152
A4,3.365,3.430,3.400,3.380,3.427,3.462,3.754,3.511,3.453,3.566,...,0.108,0.125,0.139,0.147,0.151,0.155,0.155,0.155,0.154,0.150
A5,3.377,3.443,3.412,3.394,3.438,3.458,3.753,3.517,3.461,3.562,...,0.107,0.123,0.136,0.144,0.148,0.152,0.152,0.153,0.152,0.148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
H8,3.513,3.574,3.548,3.526,3.564,3.542,3.765,3.572,3.509,3.576,...,0.109,0.117,0.125,0.131,0.134,0.138,0.138,0.141,0.142,0.140
H9,3.482,3.538,3.516,3.497,3.524,3.556,3.739,3.543,3.482,3.556,...,0.105,0.113,0.120,0.125,0.129,0.133,0.133,0.136,0.138,0.136
H10,3.349,3.409,3.380,3.363,3.405,3.430,3.675,3.457,3.396,3.481,...,0.078,0.085,0.092,0.097,0.101,0.105,0.105,0.109,0.111,0.109
H11,3.479,3.538,3.511,3.493,3.529,3.530,3.735,3.551,3.475,3.555,...,0.104,0.112,0.119,0.124,0.128,0.132,0.132,0.135,0.136,0.134


Next we reset both indexes of the two dataframes we are to merge and concatnate them together. We drop and reset indexes to ensure any past dependency is forgotten, so you must make sure your sample info dataframe and plate reader dataframe are ordered correctly where row 1...n of one dataframe referes to row 1...n of the other. 

In [20]:
sample_info.reset_index(drop=True, inplace=True)
platereader_df.reset_index(drop=True, inplace=True)
combined_df = pd.concat([sample_info, platereader_df], axis = 1) # this is called combine hotdog. 
combined_df

,UID,dppc wtf,dspepeg200 wtf,pfh wtf,ethanol wtf,water wtf,Well,Labware,Slot,dppc-ethanol-stock uL,...,950.0nm,955.0nm,960.0nm,965.0nm,970.0nm,975.0nm,980.0nm,985.0nm,990.0nm,995.0nm
0,S6_A1_03-26-2021_Right,0.000075,0.000035,0.000000,0.110526,0.889364,A1,Corning 96 Well Plate 360 µL Flat,6,30.000002,...,0.122,0.143,0.159,0.167,0.171,0.174,0.173,0.172,0.170,0.164
1,S6_A2_03-26-2021_Right,0.000075,0.000035,0.000000,0.147368,0.852522,A2,Corning 96 Well Plate 360 µL Flat,6,30.000002,...,0.119,0.138,0.154,0.162,0.166,0.170,0.169,0.169,0.167,0.162
2,S6_A3_03-26-2021_Right,0.000075,0.000035,0.000000,0.184211,0.815679,A3,Corning 96 Well Plate 360 µL Flat,6,30.000002,...,0.109,0.128,0.142,0.150,0.154,0.158,0.158,0.158,0.156,0.152
3,S6_A4_03-26-2021_Right,0.000075,0.000035,0.000000,0.221053,0.778837,A4,Corning 96 Well Plate 360 µL Flat,6,30.000002,...,0.108,0.125,0.139,0.147,0.151,0.155,0.155,0.155,0.154,0.150
4,S6_A5_03-26-2021_Right,0.000075,0.000035,0.000000,0.257895,0.741995,A5,Corning 96 Well Plate 360 µL Flat,6,30.000002,...,0.107,0.123,0.136,0.144,0.148,0.152,0.152,0.153,0.152,0.148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,S6_H8_03-26-2021_Right,0.000075,0.000035,0.003333,0.626316,0.370241,H8,Corning 96 Well Plate 360 µL Flat,6,30.000002,...,0.109,0.117,0.125,0.131,0.134,0.138,0.138,0.141,0.142,0.140
92,S6_H9_03-26-2021_Right,0.000075,0.000035,0.003333,0.663158,0.333399,H9,Corning 96 Well Plate 360 µL Flat,6,30.000002,...,0.105,0.113,0.120,0.125,0.129,0.133,0.133,0.136,0.138,0.136
93,S6_H10_03-26-2021_Right,0.000075,0.000035,0.003333,0.700000,0.296557,H10,Corning 96 Well Plate 360 µL Flat,6,30.000002,...,0.078,0.085,0.092,0.097,0.101,0.105,0.105,0.109,0.111,0.109
94,S6_H11_03-26-2021_Right,0.000075,0.000035,0.003750,0.663158,0.332982,H11,Corning 96 Well Plate 360 µL Flat,6,30.000002,...,0.104,0.112,0.119,0.124,0.128,0.132,0.132,0.135,0.136,0.134


# Step 4: Format Dataframe as csv and Upload to Drive.
We simply call a method of the dataframe while ensuring to drop the index (again to remove and previous dependency). 

***If uploading from desktop***: Just grab and upload like any other file.

***If uploading from jupyter notebook***: Save the csv anywhere with a name that is not relevant, reccomended is something like "Temp File" as you will be uploading it to google drive under a different name. Then use the Gdrive module of functions from the Prepare package. Read the README in this package to understand how to use it, it will require a certain setup of credentials. 


In [21]:
temp_hold_path = r"SynthesisInfo_PlusSpectra"
combined_df_csv = combined_df.to_csv(temp_hold_path, index=False)